## Setup

To access material for this workbook please execute the two notebook cells immediately below (e.g. use the shortcut <b>&lt;shift&gt;+&lt;return&gt;</b>). The first cell can be skipped if you are running this notebook locally and have already installed all the necessary packages. The second cell should print out "Your notebook is ready to go!"

In [ ]:
if 'pyodide_kernel' in str(get_ipython()):  # specify packages to install under JupyterLite
    raise RuntimeError("This workbook is not designed to run in JupyterLite. Please use a Colab or local install")
elif 'google.colab' in str(get_ipython()):  # specify package location for loading in Colab
    from google.colab import drive
    drive.mount('/content/drive')
    %run /content/drive/MyDrive/GARG_workshop/Notebooks/add_module_path.py
else:  # install packages on your local machine (-q = "quiet": don't print out installation steps)
    !python -m pip install -q -r https://github.com/ebp-nor/GARG/raw/main/jlite/requirements.txt

In [ ]:
# Load questions etc for this workbook
from IPython.display import SVG
import tskit
import ARG_workshop
workbook = ARG_workshop.Workbook2B()
display(workbook.setup)

### Using this workbook

This workbook is intended to be used by executing each cell as you go along. Code cells (like those above) can be modified and re-executed to perform different behaviour or additional analysis. You can use this to complete various programming exercises, some of which have associated questions to test your understanding. Exercises are marked like this:
<dl class="exercise"><dt>Exercise XXX</dt>
<dd>Here is an exercise: normally there will be a code cell below this box for you to work in</dd>
</dl>

# Workbook 2-B: pairwise inference

The simplest ARG just consists of a pair of sample nodes, linked to a set of root nodes (sometimes called a "cherry" by phylogeneticists). There are no topological differences between the local trees, just a change in root height. Here's an example visualization:

In [ ]:
import msprime
import math
ts = msprime.sim_ancestry(1, population_size=1e5, sequence_length=5e3, recombination_rate=1e-8, random_seed=2)

# Snazzy 3D code taken from https://tskit.dev/tutorials/viz.html#d-effects
tree_width, height = 100, 200
y_step = 40  # Stagger between trees (i.e. 0 for all trees in a horizontal line)
skew = 0.3  # How skewed the trees are, in radians
n = ts.num_trees
width = tree_width * n + 20 + 20  # L & R margins
angle = math.atan(y_step/tree_width)
ax_mv = y_step, (n - 1) * y_step + math.tan(skew) * (tree_width * .9)
style = f".x-axis {{transform: translate({ax_mv[0]}px, {ax_mv[1]}px) skewY(-{angle}rad)}}"
for i in range(ts.num_trees):
    style += f".tree.t{i} > .plotbox {{transform:translateY({(n - i - 1) * y_step}px) skewY({skew}rad)}}"
canvas_size = (width + y_step, height + ts.num_trees*y_step + math.tan(skew)*tree_width)
ts.draw_svg(size=(width, height), x_scale="treewise", style=style, canvas_size=canvas_size)

You will see that the distances spanned by each tree can be quite variable (we have spaced them out evenly for visualization purposes, but e.g. the last tree only spans 38 bp out of this 5000bp genome.

As we previously saw, each coalescence point (i.e. MRCA, or root of the) can be surprisingly long ago. In a typical species, we might expect a pair of haploid genomes to contain tens of thousands of different roots on each reasonably sized chromosome. In a species such as humans, hundreds of thousands, or even millions of coalescent points are responsible for causing the variation between a pair of genomes. _Pairwise_ approaches, as long as they encompass the whole genome, are therefore a surprisingly informative source of information for inferring population history.

However, if we try to pair evey haploid genome with every other haploid genome, the scaling becomes quadratic, and therefore infeasible for large datasets. Fortunately, most organisms are *diploid*, and therefore it is quite reasonable to compare the two parental genomes in a single individual (after all, we know that they are likely to be from the same population, unless there has been very recent admixture). We can even do this for a large number of different individuals, for comparison.

This is the basic premise pioneered by Heng Li and Richard Durbin, in their classic [2011 paper](https://www.nature.com/articles/nature10231), associated with their [PSMC](https://github.com/lh3/psmc) software. Several elaborations on the basic method have been made, and there has been a burst of recent activity in the area.

In [ ]:
# Execute code block with <shift>+Return to display question; type and press return, or click on the buttons to answer
workbook.question("smc")

## The idea behind PSMC

A major reason behind the success of PSMC-like methods is that there is no complex topology to infer. All that needs doing is to infer the (unknown) *time* of the MRCAs as we go along the genome. Compared to ARG inference methods that estimate topology (e.g. _tsinfer_) separately from node times (_tsdate_), it is basically equivalent to running only the second, dating step; as we'll see in the next workbook, that tends to be very fast.

If the SMC approximation holds, this allows a classic way to estimate hidden or unknown variables, using Hidden Markov Models (HMMs).



In [ ]:
# Execute code block with <shift>+Return to display question; type and press return, or click on the buttons to answer
workbook.question("hmm_psmc")

## Classic PSMC-like software

The original _PSMC_ software is still  and widely used, although it's now recommended to move to [MSMC2](https://github.com/stschiff/msmc2), e.g. see [this tutorial](https://evomics.org/learning/population-and-speciation-genomics/2020-population-and-speciation-genomics/demography-psmc-msmc/) by Richard Durbin, from whose group these two methods emerged.

We may encounter some of the bleeding edge methods like [Phlash](https://pubmed.ncbi.nlm.nih.gov/38585997/), [cobraa](https://www.biorxiv.org/content/10.1101/2024.03.24.586479v1.full.pdf), and [Gamma-SMC](https://genome.cshlp.org/content/33/7/1023) in the session tomorrow. However, for simple teaching purposes, this workbook will use a [python-only reimplementation](https://github.com/tulerpetontidae/psmc-python) of PSMC, which can take tree sequences as input. Thanks to Artem Lomakin for making this available: a copy of the code has been placed in the `pmsc_python` folder in the current directory.

We will follow the [psmc-python example notebook](https://github.com/tulerpetontidae/psmc-python/blob/main/example-tskit.ipynb) and download some data stored in _tskit_ format from the [Unified genealogy](https://www.science.org/doi/10.1126/science.abi8264). We will then re-infer MRCAs for individuals using PSMC.

In [ ]:
import urllib.request
import os
chrom = "17_q"  # Q arm of chr 17
file = f"hgdp_tgp_sgdp_chr{chrom}.dated.trees.tsz"
url = f"https://zenodo.org/records/5495535/files/{file}"
path = os.path.join("data", file)
if not os.path.exists(path):
    with workbook.download(url) as t:
        urllib.request.urlretrieve(url, filename=path, reporthook=t.update_to)

In [ ]:
import tszip
import json

# Open and pick some selected genomes
ts = tszip.decompress(path).trim()  # remove the missing flanking regions
# Make a dict of 5 individuals from different populations, taken from the HGDP data
# Feel free to pick some others
use_pops = {"Biaka", "Mbuti", "San", "Japanese", "Uygur", "French"}
all_pops = set()
individuals = {}
for i in ts.individuals():
    metadata = json.loads(i.metadata.decode())
    if metadata.get("sample", "").startswith("HGDP"):
        # This is an HGDP individual (could also try with SGDP or 1000genomes)
        pop = ts.population(ts.node(i.nodes[0]).population)
        pop_metadata = json.loads(pop.metadata.decode())
        name = pop_metadata["name"]
        all_pops.add(name)
        if name in use_pops:
            print(f"Using {metadata['sample']} from {pop_metadata}")
            individuals[name + "-" + pop_metadata["region"]] = i.id
            use_pops.remove(name)
print(f"Other populations you could choose from: {all_pops - use_pops}")
print(f"Couldn't find {use_pops} in {all_pops}" if len(use_pops) else f"Individual IDs: {individuals}")


To get reasonable results, you would normally run PSMC on a large number of chromosomes, simultaneously. This can take some time, although more modern software packages can speed this up (for real analysis, you are recommended to use something like Phlash rather than psmc-python, which is even slower than the original psmc).

The original PSMC algorithm uses a bespoke FASTA-like format in which the entire genome binned into 100bp windows, and the windows are classified according to whether they have a heterozygous site or not. This rather customised discretisation is followed in the implementation below (when converting a tree sequence into an input data array to save). Other PSMC-like approaches may differ in this respect.

In [ ]:
from psmc_python.utils import process_ts
import numpy as np

# get an `x` object for input into psmc-python: normally this is of shape
# num_chromosomes x num_windows. Here we are just using a single chromosome, for speed.
for name in individuals.keys():
    x = process_ts(ts, individual=individuals[name], progress=True)
    outfile = os.path.join("data", name + ".npy")
    print(f"Saved {x.shape[1]} 100bp windows into '{outfile}' for {x.shape[0]} chromosome(s)")
    np.save(outfile, x)

PSMC uses discrete time bins, and for speed, merges some of the time bins together. The `pattern` argument specifies how to do this and the Li & Durbin recommend a [particular incantation for the autosomes](https://github.com/tulerpetontidae/psmc-python/issues/5), which we will use here without further comment. There has been a recent move towards reducing the effect of discrete bins, either by using a parameterised distribution (e.g. beta or gamma) or by sampling time bin boundaries themselves.

In [ ]:
# The following function is also defined in workbook.run_psmc, which allows running in windows
# See https://bobswinkels.com/posts/multiprocessing-python-windows-jupyter/
def run_psmc(params):
    from psmc_python.model import PSMC

    data_file = params[0]
    data = np.load(data_file)
    n_iter = params[1] if len(params) > 1 else 10
    start_window = params[2] if len(params) > 2 else 0
    end_window = params[3] if len(params) > 3 else data.shape[1]
    data = data[:, start_window: end_window]
    print(f"Using {data.shape[1]} 100bp windows for inference", flush=True)
    theta0 = np.sum(data) / (data.shape[0] * data.shape[1])
    rho0 = theta0 / 5

    # initialise new instance and run EM - do not show a progress bar, as it doesn't
    # work when run in parallel
    psmc_model = PSMC(t_max=15, n_steps=64, pattern='1*4+25*2+1*4+1*6', progress_bar=None)
    psmc_model.param_recalculate()

    initial_params = [theta0, rho0, 15] + [1.] * (psmc_model.n_free_params - 3)
    bounds = [(1e-4, 1e-1), (1e-5, 1e-1), (12, 20)] + [(0.1, 10)] * (psmc_model.n_free_params - 3)
    name = data_file.replace(".npy", "")
    loss_list, params_history = psmc_model.EM(initial_params, bounds, x=data, n_iter=n_iter, name=name)
    psmc_model.save_params(name + ".json")
    

We can now run the EM algorithm. This runs 5 iterations of the EM algorithm for each individual. Each iteration can
take about 5 minutes, so may may take 25 minutes for each individual. At least we can parallelize all 6 individuals on separate cores, which is what the `multiprocessing.Pool` invocation does.

Go and have a cup of tea or coffee while it runs, or read up on the [original PSMC paper](https://www.nature.com/articles/nature10231) (short and readable) or in a blog post [discussing population structure](https://www.molecularecologist.com/2016/05/18/opening-pandoras-box-psmc-and-population-structure/). Note that [a recent preprint](https://www.biorxiv.org/content/10.1101/2024.03.24.586479v1.full) describing the cobraa software indicates that there is more information available in the $T_\mathrm{MRCA}$s that could help distinguish between changes in ancient structure and true changes in population size. We may wish to discuss the implications of this in a collaborative session.

In [ ]:
from multiprocessing import Pool
# Only choose e.g. 100_000 x 100bp windows. You can change this (or omit it) to include
# more of the genome in order to increase accuracy, but it will take longer to run
start_window_idx = 100_000
end_window_idx = 500_000
task_params = [
    (
        os.path.join("data", name) + ".npy",
        5,  # Number of EM iterations
        start_window_idx,
        end_window_idx
    ) for name in individuals.keys()
]

with Pool(processes=6) as pool:  # set to between 1 and the number of CPU cores on your machine
    _ = pool.map(workbook.run_psmc, task_params)


In [ ]:
import matplotlib.pyplot as plt
from psmc_python.plot import plot_history
from psmc_python.model import PSMC

fig, axs = plt.subplots(1,1,figsize=(12,6))
psmc = PSMC()
afr_colours = iter(('tomato', 'orangered', 'darkorange'))
other_colours = iter(('skyblue', 'dodgerblue', 'cornflowerblue'))
for subpop, sample_id in individuals.items():
    color=next(afr_colours if subpop.endswith("AFRICA") else other_colours)
    psmc.load_params(os.path.join("data", subpop + ".json"))
    plot_history(psmc, th=20, axs=axs, label=subpop + f" (sample {sample_id})", color=color)
        
axs.set_ylim(0,2.5e4);

According to Heng Li, the PSMC author, ["Not correcting for low coverage is the most common pitfall when using PSMC."](https://github.com/lh3/psmc). However, these data are based on WGS should should be robust to this

## Other useful HMMs

On the topic of HMMs, it's worth mentioning another now-classic HMM: the Li & Stephens HMM. There will be a talk on this later.